# Balanced Risk Set Matching (Yunfei et al., 2011) Review

This is not my field of study, so there might be some misinterpretation that may occur on this review. This is a study of treatment, cystoscopy, and hydrodistention, given in response to the symptoms of the chronic, nonlethal disease interstitial cystitis. The idea of the journal is to match the treatment of the patient with a similar history of symptoms who have different times receiving their treatment. It is described as time $t$ for when the patient received their treatment.

## Matching To Achieve Comparable Histories

If patient $m$ received the treatment at time $T_m$, then compare the response of this patient to a patient who did not recieve the treatment up to time $T_m$. Or at least, one would like treated and control groups whose aggregate distributuions of symtoms were similar.

Let $S$ be matched pairs, $s = 1$, $\dots$, $S$, containing $2S$ distinct patients, such that the treated patient in pair $s$ recieved the treatment at time $T_s$, and the control in pair $s$ either did not receive the treatment at all or recieved it stricly after $T_s$.

## Optimal Balanced Matching

The optimal pair matching problem involves a finite set $A = { \alpha_1, \dots, \alpha_M }$ called units, a subset $T \subseteq A$ called treated units, and a subset of their direct product $\varepsilon \subseteq T \times A$ called edges. If the pair $e = (\alpha_p, \alpha_q)$ is an edge $e \in \varepsilon$, then it is permitted to match $\alpha_p$ to $\alpha_q$, but if $e \not\in \varepsilon$, then this match is forbidden.

$T \subseteq A$ consists of all patients in the sample who eventually received the treatment. The pair $e = (\alpha_p, \alpha_q)$ is an edge $e \in \varepsilon$ if $\alpha_p$ received the treatment, say at time $T_p$, and $\alpha_q$ either never received the treatment or received it strictly after time $T_p$. Set $\varepsilon$ may exclude certain pairs for additional reasons, such as being too far apart on an important covariate. Without a risk set in a study in which everyone receives either treatment or control immediately at baseline; then $T$ contains the treated subjects, $A - T$ contains the controls, and $\varepsilon \subseteq T \times (A - T)$ requires treated subects to be matched to untreated controls.